In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
#to get training and test data
from sklearn.model_selection import train_test_split
# to work on images 
from PIL import Image

In [2]:
# Path to the image folder -----------------------------------change this accordingly 
image_folder = r"T:\TOSHITH\PROGRAMMING\music_genere_classification\image_data_processed_rgb"

In [3]:
#loading the data into numpy array 
image_list = []
label_list = []

#class_names = os.listdir(image_folder) #names of the subfolder
class_names=sorted(os.listdir(image_folder))

for class_name in class_names:
    class_path=os.path.join(image_folder, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        try:
            img = Image.open(img_path)
            #img = img.resize((200,200))  # Resize to desired shape
            img_array = np.array(img)
            image_list.append(img_array)

            label_list.append(class_name)
        except Exception as e:
            print(f"Failed to load {img_path}: {e}")

# Convert to NumPy arrays
images = np.array(image_list)
labels = np.array(label_list)

print("Shape of images:", images.shape)
print("Shape of labels:", labels.shape)

Shape of images: (999, 288, 432, 3)
Shape of labels: (999,)


In [ ]:
from sklearn.preprocessing import LabelBinarizer

# Assuming labels is a 1D array of strings (genre names)
lb = LabelBinarizer()
labels_encoded = lb.fit_transform(labels)

print("Shape of one-hot encoded labels:", labels_encoded.shape)

Shape of one-hot encoded labels: (999, 10)


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [25]:
# creating a 70|15|15 training test and validation split 

# 15% split of test data
image_temp, image_test, label_temp, label_test = train_test_split(
    images, labels_encoded, test_size=0.15, random_state=45, stratify=labels
)

# 70 and 15 percent split of training and validation 0.1765=15/85  
image_train, image_val, label_train, label_val = train_test_split(
    image_temp, label_temp, test_size=0.1765, random_state=6, stratify=label_temp
)

print("no of images for training",len(image_train))
print("no of images for testing",len(image_test))
print("no of images for validation",len(image_val))
type(image_test)
image_test.shape

no of images for training 699
no of images for testing 150
no of images for validation 150


(150, 288, 432, 3)

In [26]:
image_train, image_test, image_val = image_train/255, image_test/255, image_val/255

In [27]:
image_test.shape[1:4]

(288, 432, 3)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models

#input to the model is of size 200x200
 
# THIS IS THE FIRST HALF 
# CONVOLUTION LAYER 

model = models.Sequential()

model.add(layers.Conv2D(64,(3,3), activation='relu', padding='same', input_shape=image_test.shape[1:4]))
model.add(layers.Conv2D(64,(3,3), activation='relu', padding='same',))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(32,(3,3), activation='relu', padding='same',))
model.add(layers.Conv2D(32,(3,3), activation='relu', padding='same',))
model.add(layers.MaxPooling2D((2,2)))


model.add(layers.Conv2D(32,(3,3), activation='relu', padding='same',))

# THIS IS THE SECOND HALF 
# THE DENSE LAYER 
model.add(layers.Flatten())

# Dropout after flattening to prevent over fitting 
model.add(layers.Dropout(0.5))

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))  # Dropout between dense layers

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.2))  # Optional: can be adjusted/removed

# Final layer for binary classification
model.add(layers.Dense(10, activation='softmax'))  # Use sigmoid for binary classification

c:\Users\toshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 288, 432, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 288, 432, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 144, 216, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 144, 216, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 144, 216, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 72, 108, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 72, 108, 32)    │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 248832)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 248832)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │    31,850,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,930,762 (121.81 MB)

 Trainable params: 31,930,762 (121.81 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#compile the model
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=["accuracy"])

#implementing for early stopping
import tensorflow 
early_stopping=tensorflow.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
model_history = model.fit(
    image_train, label_train,
    validation_data=(image_val, label_val),  # use this instead of validation_split
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping]
)

Epoch 1/100
28/44 ━━━━━━━━━━━━━━━━━━━━ 14s 936ms/step - accuracy: 0.0624 - loss: 2.4417

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/gradient_tape/sequential_2_1/conv2d_12_1/convolution/Conv2DBackpropFilter defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a primitive, in file tensorflow/core/kernels/mkl/mkl_conv_grad_filter_ops.cc:685
	 [[{{node StatefulPartitionedCall/gradient_tape/sequential_2_1/conv2d_12_1/convolution/Conv2DBackpropFilter}}]] [Op:__inference_one_step_on_iterator_3741]